# 07 - Generate statistics CSVs
Creating a separate notebook to calculate the statistics all in one pass so that I can complete a representative set of CSVs for each instrument class.

### Statistics for QARTOD tests in production

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import re
import warnings
warnings.filterwarnings("ignore")

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

# Import qartod_testing project functions
from qartod_testing.data_processing import ooinet_gold_copy_request, get_test_parameters, parse_qartod_executed, qartod_summary_expanded

# Import OOI library functions
from ooi_data_explorations.common import merge_frames

In [2]:
# Setup parameters needed to request data
refdes = "GA01SUMO-RID16-03-CTDBPF000"
method = "recovered_inst"
stream = "ctdbp_cdef_instrument_recovered"

# Site, node, and sensor info from deconstructed reference designator
# [site, node, sensor] = refdes.split('-', 2)

In [3]:
# Routine in data_processing module from this project to download the gold copy THREDDs datasets
# Variable 'files' contains list of catalog URLs for downloaded datasets 
files = ooinet_gold_copy_request(refdes, method, stream)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Load expected results data from external data folder
folder_path = os.path.join(os.path.abspath('../data/external'), method, stream, refdes)
expected_files = glob.glob(folder_path+'/*.nc')
expected_files.sort() # sorts local test files in alphanumeric order

#### Gross range test statistics

In [ ]:
def collect_statistics(file_paths, test_name):
    """
    Calls other functions to calculate statistics from a set of files and a name of a QARTOD test. The statistics are organized in a DataFrame.
    
    Parameters:
    -----------
        file_paths: list of paths to each file that will have statistics calculated. File names must include "deployment00##".
        test_name: string of QARTOD test name, i.e. "gross_range", "climatology".
        
    Returns:
    --------
        statistics: Pandas DataFrame containing statistics on each parameter with a QARTOD test in order of deployment number, then statistics of the full record.
        
    Version 16 Aug 2023, Kylene M Cooley    
    """
    
    # Initialize empty dictionary for statistics
    statistics = {}

    for m, _ in enumerate(file_paths):
        file = file_paths[m]

        # get deployment from current file, then open local test and expected test datasets
        deployment = re.findall('deployment00[0-2][0-9]', file)[0][-2:]
        file_ds = xr.open_dataset(file)

        # Get parameters that have QARTOD executed from expected test dataset
        test_parameters = get_test_parameters(file_ds)
        parameters = list(test_parameters.keys())

        # Separate QARTOD test flags in expected test dataset by QARTOD test name
        file_ds = parse_qartod_executed(file_ds, parameters)

        # Update summary statistics dictionary for each deployment, then for all deployments
        print("Evaluating statistics on QARTOD flags for deployment "f"{deployment}")
        summary_results = qartod_summary_expanded(file_ds, parameters, deployment, test_name)
        statistics.update({f"{m}" : summary_results })

        # Add entry to summary statistics for full data record after last file
        if file == file_paths[-1]:
            # Open all expected data files and create merged full dataset
            merged_ds = [xr.open_dataset(single_file) for single_file in file_paths]
            merged_ds = merge_frames(merged_ds)
            deployment = "all"

            # Summary of flags from merged dataset
            print("Evaluating statistics on QARTOD flags for all deployments")
            merged_ds = parse_qartod_executed(merged_ds, parameters)
            summary_results = qartod_summary_expanded(merged_ds, parameters, deployment, test_name)
            statistics.update({ "all" : summary_results })

    # Create data frame from dictionary and check contents
    statistics = pd.DataFrame.from_dict(statistics, orient='index')
    statistics = statistics.set_index('deployment')
    return statistics

In [ ]:
gross_range_stats = collect_statistics(expected_files, "gross_range")
gross_range_stats

In [ ]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
gross_range_stats.to_csv(folder_path+f"/gross_range-{refdes}-flag_statistics.csv", na_rep='NaN', mode='a')

#### Climatology test statistics